# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#pindah ke direktori yang digunakan
%cd '/content/drive/MyDrive/1 Pengajaran Pendidikan/Kuliah/SI_BISAAI_NLP'

/content/drive/MyDrive/1 Pengajaran Pendidikan/Kuliah/SI_BISAAI_NLP


#1.Import Library




In [ ]:
!pip install dataframe-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install varname

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras-metrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
import pandas as pd
import re
import tensorflow as tf
import time
import multiprocessing
import io
import gensim
import numpy as np
import matplotlib.pyplot as plt
import keras_metrics as km
import pickle
import keras
import nltk

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
#from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from varname import nameof


nltk.download('punkt')
from nltk.tokenize import word_tokenize


from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#2.Load Data

In [ ]:
#simpan path dataset
path_data = "ulasan-20.csv"
 
#read dataset
data_dataset = pd.read_csv(path_data, sep=",", header=[0], encoding="UTF-8")

#3.Implementasi Preprocessing

##3.1. Casefolding

In [ ]:
#mengambil data ulasan
data_content = data_dataset ['ulasan']
print(data_content[:4])

0    Dalam beberapa Bulan terakhir saya sempat ke s...
1    Salah satu tempat yang harus dikunjungi saat m...
2    Recommended untuk yg ingin melihat keindahan o...
3    Air terjun Sekumpul Sekarang menjadi lebih bai...
Name: ulasan, dtype: object


In [ ]:
#casefolding
data_casefolding = data_content.str.lower()
data_casefolding.head()

0    dalam beberapa bulan terakhir saya sempat ke s...
1    salah satu tempat yang harus dikunjungi saat m...
2    recommended untuk yg ingin melihat keindahan o...
3    air terjun sekumpul sekarang menjadi lebih bai...
4    air terjun, pemandangan bagus dan penduduk yan...
Name: ulasan, dtype: object

##3.2. Filtering

In [ ]:
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|
\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''',
 " ", tweet) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  
#hapus simbol'[!#?,.:";@()-_/\']'
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)


NameError: ignored

In [ ]:
data_filtering

0     dalam beberapa bulan terakhir saya sempat ke s...
1     salah satu tempat yang harus dikunjungi saat m...
2     recommended untuk yg ingin melihat keindahan o...
3     air terjun sekumpul sekarang menjadi lebih bai...
4     air terjun  pemandangan bagus dan penduduk yan...
5     pada saat musim hujan kedua air terjun ini aka...
6     air terjun terbaik yang pernah saya lihat  lok...
7     air terjun sekumpul berjarak sekitar   km dari...
8     air terjun yang indah ini terletak di desa lem...
9     yeyy   bagi yang suka wisata alam   sekumpul w...
10    air terjun sekumpul adalah salah satu air terj...
11    kereeeennnn abiiiisssss\ndengan beberapa air t...
12    pemandangan yang luar biasa  pilihan tepat bag...
13    kami menginap di salah satu hotel di kawah gun...
14    pemandangannya sangat indah dan udaranya juga ...
15    air terjun \nmendengar istilah tersebut pasti ...
16    pemandangan yang indah beserta dengan sekumpul...
17    sekumpul adalah nama air terjun yang berad

## 3.3. Tokenization

In [ ]:
#tokenization

data_tokens = [word_tokenize(line) for line in data_filtering]
print(data_tokens)

[['dalam', 'beberapa', 'bulan', 'terakhir', 'saya', 'sempat', 'ke', 'sana', 'kali', 'sebelum', 'dan', 'sesudah', 'lahar', 'dingin', 'ke', 'luar', 'pemandangan', 'yang', 'paling', 'mengesankan', 'bagi', 'saya', 'saat', 'melihat', 'view', 'dari', 'desa', 'sidemang', 'juga', 'pura', 'lempuyang', 'dengan', 'gate', 'yang', 'view', 'gunung', 'agungnya', 'sangat', 'terlihat', 'jelas', 'dan', 'indah', 'sekali'], ['salah', 'satu', 'tempat', 'yang', 'harus', 'dikunjungi', 'saat', 'mengunjungi', 'bali', 'masuknya', 'gratis', 'dan', 'juga', 'mudah', 'ditemukan', 'dengan', 'google', 'map'], ['recommended', 'untuk', 'yg', 'ingin', 'melihat', 'keindahan', 'ombak', 'dalam', 'lake', 'kecil', 'sangat', 'menarik', 'dan', 'unik', 'cocok', 'untuk', 'photo', 'prewedding'], ['air', 'terjun', 'sekumpul', 'sekarang', 'menjadi', 'lebih', 'baik', 'lagi', 'terawat', 'dengan', 'baik', 'jadi', 'semakin', 'indah', 'tempat', 'ini', 'dekat', 'dengan', 'air', 'terjun', 'fiji', 'di', 'lemukih'], ['air', 'terjun', 'peman

##3.4. Konversi Slangword

In [ ]:
#slang word
path_dataslang = open("kamus kata baku 1.csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokens:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
print(data_formal)
len_data_formal

[['dalam', 'beberapa', 'bulan', 'terakhir', 'saya', 'sempat', 'ke', 'sana', 'kali', 'sebelum', 'dan', 'sesudah', 'lahar', 'dingin', 'ke', 'luar', 'pemandangan', 'yang', 'paling', 'mengesankan', 'bagi', 'saya', 'saat', 'melihat', 'pemandangan', 'dari', 'desa', 'sidemang', 'juga', 'pura', 'lempuyang', 'dengan', 'gate', 'yang', 'pemandangan', 'gunung', 'agungnya', 'sangat', 'terlihat', 'jelas', 'dan', 'indah', 'sekali'], ['salah', 'satu', 'tempat', 'yang', 'harus', 'dikunjungi', 'saat', 'mengunjungi', 'bali', 'masuknya', 'gratis', 'dan', 'juga', 'mudah', 'ditemukan', 'dengan', 'google', 'map'], ['recommended', 'untuk', 'yang', 'ingin', 'melihat', 'keindahan', 'ombak', 'dalam', 'lake', 'kecil', 'sangat', 'menarik', 'dan', 'unik', 'cocok', 'untuk', 'photo', 'prewedding'], ['air', 'terjun', 'sekumpul', 'sekarang', 'menjadi', 'lebih', 'baik', 'lagi', 'terawat', 'dengan', 'baik', 'jadi', 'semakin', 'indah', 'tempat', 'ini', 'dekat', 'dengan', 'air', 'terjun', 'fiji', 'di', 'lemukih'], ['air', 

41

## 3.5. Stopword Removal

In [ ]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
print(pd.DataFrame(default_stop_words)[:10])

         0
0      ada
1   adalah
2   adanya
3   adapun
4     agak
5  agaknya
6     agar
7     akan
8  akankah
9    akhir


In [ ]:
len(data_notstopword)

41

In [ ]:
data_notstopword

[['kali',
  'lahar',
  'dingin',
  'pemandangan',
  'mengesankan',
  'pemandangan',
  'desa',
  'sidemang',
  'pura',
  'lempuyang',
  'gate',
  'pemandangan',
  'gunung',
  'agungnya',
  'indah'],
 ['salah',
  'dikunjungi',
  'mengunjungi',
  'bali',
  'masuknya',
  'gratis',
  'mudah',
  'ditemukan',
  'google',
  'map'],
 ['recommended',
  'keindahan',
  'ombak',
  'lake',
  'menarik',
  'unik',
  'cocok',
  'photo',
  'prewedding'],
 ['air',
  'terjun',
  'sekumpul',
  'terawat',
  'indah',
  'air',
  'terjun',
  'fiji',
  'lemukih'],
 ['air',
  'terjun',
  'pemandangan',
  'bagus',
  'penduduk',
  'ramah',
  'lokasi',
  'bersih',
  'bawa',
  'pakain',
  'ganti',
  'berenang'],
 ['musim',
  'hujan',
  'air',
  'terjun',
  'memiliki',
  'warna',
  'air',
  'berbeda',
  'sebelah',
  'kiri',
  'airnya',
  'berwarna',
  'jernih',
  'bersih',
  'berasal',
  'mata',
  'air',
  'letaknya',
  'diatas',
  'perbukitan',
  'desa',
  'sekumpul',
  'air',
  'terjun',
  'sebelah',
  'kanan',
  '

##3.6. Stemming

In [ ]:
white_list = ["bali"] #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i

factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      if(word not in white_list):
        word = ind_stemmer.stem(word)
      if(len(word)>3):
        temp.append(word)
    return temp

reviews = [stemmer (line) for line in data_notstopword]
print(reviews)

NameError: ignored

In [ ]:
#coba ganti ke pandas
data_bersih = pd.Series(reviews)

In [ ]:
#coba simpan ke csv
data_bersih.to_csv('databersih.csv')

#4.Konversi Kalimat

In [ ]:
#Pembuatan Kamus kata
t  = Tokenizer()
fit_text = reviews
t.fit_on_texts(fit_text)

#Pembuatan Id masing-masing kata
sequences = t.texts_to_sequences(reviews)

#hapus duplikat kata yang muncul
list_set_sequence = [list(dict.fromkeys(seq)) for seq in sequences]

#mencari max length sequence
def FindMaxLength(lst): 
    maxList = max((x) for x in lst) 
    maxLength = max(len(x) for x in lst ) 
    return maxList, maxLength 
      
# Driver Code 
max_seq, max_length_seq = FindMaxLength(list_set_sequence)
jumlah_index = len(t.word_index) +1

print('jumlah index : ',jumlah_index,'\n')
print('word_index : ',t.word_index,'\n')
print('index kalimat asli     : ', sequences,'\n')
print('kalimat tanpa duplikat : ',list_set_sequence,'\n')
print('panjang max kalimat : ', max_length_seq,'kata','\n')
# print('kalimat terpanjang setelah dihapus duplikat : ', max_seq,'\n')

count_word = [len(i) for i in list_set_sequence]
print('list panjang kalimat : ', count_word)
max_len_word = max(count_word)
print(max_len_word)

jumlah index :  374 

word_index :  {'terjun': 1, 'pantai': 2, 'pandang': 3, 'indah': 4, 'bagus': 5, 'jalan': 6, 'kumpul': 7, 'tangga': 8, 'tarik': 9, 'kesini': 10, 'anak': 11, 'kesan': 12, 'desa': 13, 'lokasi': 14, 'suasana': 15, 'nyaman': 16, 'salah': 17, 'kunjung': 18, 'bersih': 19, 'motor': 20, 'lewat': 21, 'bayar': 22, 'banget': 23, 'orang': 24, 'singaraja': 25, 'wisatawan': 26, 'kali': 27, 'unjung': 28, 'saran': 29, 'alam': 30, 'barang': 31, 'dingin': 32, 'bali': 33, 'awat': 34, 'lemukih': 35, 'renang': 36, 'milik': 37, 'kiri': 38, 'sungai': 39, 'langsung': 40, 'parkir': 41, 'tiket': 42, 'sana': 43, 'rekomendasi': 44, 'turun': 45, 'enak': 46, 'makan': 47, 'kotor': 48, 'masuk': 49, 'mudah': 50, 'recommended': 51, 'cocok': 52, 'duduk': 53, 'warna': 54, 'jernih': 55, 'letak': 56, 'kanan': 57, 'baik': 58, 'lihat': 59, 'lelah': 60, 'segar': 61, 'wisata': 62, 'suka': 63, 'nama': 64, 'sejuk': 65, 'hotel': 66, 'mandi': 67, 'pagi': 68, 'nikmat': 69, 'hati': 70, 'foto': 71, 'legi': 72, 'dr

In [ ]:
min_len_word=min(count_word)
print(min_len_word)

6


In [ ]:
print ("Original list is : " + str(list_set_sequence[0])) 

Original list is : [27, 131, 32, 3, 12, 13, 132, 133, 134, 135, 78, 136, 4]


## 4.1. Padding

In [ ]:
#Padding
from keras_preprocessing.sequence import pad_sequences
padding= pad_sequences([list(list_set_sequence[i]) for i in range(len(list_set_sequence))],
maxlen= max_len_word, padding='pre')
padding[:10]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,  27, 131,  32,   3,  12,  13,
        132, 133, 134, 135,  78, 136,   4],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  17,  18,
         28,  33,  49, 137,  50, 138, 139],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  51,   4,
        140, 141,   9, 142,  52, 143

# TOPIC 5 - FEATURE EXTRACTION

##Load Hasil Preprocessing & Join


In [ ]:

reviews2 = [" ".join(r) for r in reviews]


##Convectorizer

In [ ]:
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(reviews2)
print(vectorizer.vocabulary_ )
print(X_vec.todense())

{'kali': 136, 'lahar': 173, 'dingin': 78, 'pandang': 242, 'kesan': 154, 'desa': 75, 'sidemang': 302, 'pura': 264, 'lempuyang': 180, 'gate': 94, 'gunung': 99, 'agung': 1, 'indah': 114, 'salah': 279, 'kunjung': 171, 'unjung': 359, 'bali': 24, 'masuk': 206, 'gratis': 97, 'mudah': 220, 'temu': 324, 'google': 96, 'recommended': 269, 'ombak': 236, 'lake': 174, 'tarik': 319, 'unik': 358, 'cocok': 61, 'photo': 257, 'prewedding': 262, 'terjun': 330, 'kumpul': 170, 'awat': 21, 'fiji': 88, 'lemukih': 181, 'bagus': 22, 'duduk': 82, 'ramah': 266, 'lokasi': 189, 'bersih': 45, 'bawa': 31, 'pakain': 241, 'ganti': 93, 'renang': 272, 'musim': 221, 'hujan': 112, 'milik': 211, 'warna': 366, 'beda': 34, 'belah': 36, 'kiri': 158, 'jernih': 128, 'asal': 14, 'mata': 207, 'letak': 182, 'atas': 18, 'bukit': 49, 'kanan': 137, 'keruh': 153, 'coklat': 62, 'sungai': 311, 'sungguh': 312, 'baik': 23, 'lihat': 185, 'susah': 316, 'jangkau': 122, 'motor': 219, 'lelah': 179, 'kesini': 156, 'lewat': 183, 'anak': 9, 'tangg

##Bag of Word 

### Building the vocabulary

In [ ]:
set_of_words = set()
for sentence in reviews2:
    for word in sentence.split():
        set_of_words.add(word)
vocab = list(set_of_words)
print(vocab)

['bayar', 'komentar', 'bentuk', 'galak', 'desember', 'batas', 'turis', 'kecewa', 'terjun', 'musim', 'robok', 'kota', 'daya', 'udara', 'hangat', 'complaint', 'staff', 'cocok', 'tiga', 'kanan', 'sekian', 'jaja', 'agung', 'lintas', 'suruh', 'attitude', 'selam', 'keruh', 'coklat', 'pagi', 'trek', 'kondisi', 'wahhhh', 'capek', 'sekar', 'alir', 'ribu', 'gigit', 'salah', 'daerah', 'selfi', 'kunjung', 'pikir', 'sunset', 'haha', 'barang', 'tmptnya', 'obat', 'tegalalang', 'yeyy', 'pasar', 'titip', 'unjung', 'minyak', 'ajar', 'motor', 'nama', 'bersih', 'turun', 'kotor', 'ganggu', 'mobil', 'kitchen', 'kenal', 'lempuyang', 'pakain', 'photo', 'tolak', 'monumen', 'obyek', 'menang', 'jalur', 'buleleng', 'coba', 'sanur', 'tipu', 'kitchennya', 'cinta', 'libur', 'lelah', 'hujan', 'sana', 'dekat', 'nafas', 'tour', 'pantai', 'travel', 'jembatan', 'viewnya', 'casing', 'mampir', 'kerang', 'lukis', 'nyaman', 'gate', 'ciprat', 'food', 'pecah', 'atus', 'wilayah', 'suka', 'duduk', 'anti', 'belanja', 'kembar', 'd

### Fetching the position of each word in the vocabulary

In [ ]:
position = {}
for i, token in enumerate(vocab):
    position[token] = i
print(position)

{'bayar': 0, 'komentar': 1, 'bentuk': 2, 'galak': 3, 'desember': 4, 'batas': 5, 'turis': 6, 'kecewa': 7, 'terjun': 8, 'musim': 9, 'robok': 10, 'kota': 11, 'daya': 12, 'udara': 13, 'hangat': 14, 'complaint': 15, 'staff': 16, 'cocok': 17, 'tiga': 18, 'kanan': 19, 'sekian': 20, 'jaja': 21, 'agung': 22, 'lintas': 23, 'suruh': 24, 'attitude': 25, 'selam': 26, 'keruh': 27, 'coklat': 28, 'pagi': 29, 'trek': 30, 'kondisi': 31, 'wahhhh': 32, 'capek': 33, 'sekar': 34, 'alir': 35, 'ribu': 36, 'gigit': 37, 'salah': 38, 'daerah': 39, 'selfi': 40, 'kunjung': 41, 'pikir': 42, 'sunset': 43, 'haha': 44, 'barang': 45, 'tmptnya': 46, 'obat': 47, 'tegalalang': 48, 'yeyy': 49, 'pasar': 50, 'titip': 51, 'unjung': 52, 'minyak': 53, 'ajar': 54, 'motor': 55, 'nama': 56, 'bersih': 57, 'turun': 58, 'kotor': 59, 'ganggu': 60, 'mobil': 61, 'kitchen': 62, 'kenal': 63, 'lempuyang': 64, 'pakain': 65, 'photo': 66, 'tolak': 67, 'monumen': 68, 'obyek': 69, 'menang': 70, 'jalur': 71, 'buleleng': 72, 'coba': 73, 'sanur': 

### Creating a matrix to hold the Bag of Words representation

In [ ]:
bow_matrix = np.zeros((len(reviews2), len(vocab)))

In [ ]:
for i, preprocessed_sentence in enumerate(reviews2):
    for token in preprocessed_sentence.split():   
        bow_matrix[i][position[token]] = bow_matrix[i][position[token]] + 1

In [ ]:
bow_matrix

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

##N-gram

### Unigram, Bigram, Trigram

In [ ]:
vectorizer_ngram_range = CountVectorizer(analyzer='word', ngram_range=(1,3))
bow_matrix_ngram = vectorizer_ngram_range.fit_transform(reviews2)

In [ ]:
print(vectorizer_ngram_range.get_feature_names())
print(bow_matrix_ngram.toarray())

['abiiiisssss', 'abiiiisssss terjun', 'abiiiisssss terjun keren', 'agung', 'agung indah', 'ajar', 'ajar mahal', 'ajar mahal alam', 'akses', 'akses terjun', 'akses terjun mudah', 'alam', 'alam buruk', 'alam daya', 'alam daya tarik', 'alam kumpul', 'alam kumpul waterfall', 'alam panorama', 'alam panorama alam', 'alam sana', 'alam sana coba', 'alami', 'alami kunjung', 'alami kunjung kesana', 'alami suasana', 'alami suasana tenang', 'alir', 'alir sungai', 'alir sungai bersih', 'ambil', 'ambil barang', 'ambil barang barang', 'ambil nafas', 'ambil nafas kena', 'ampun', 'ampun lewat', 'ampun lewat ratus', 'anak', 'anak anak', 'anak anak ganggu', 'anak ganggu', 'anak ganggu makan', 'anak tangga', 'anak tangga bayar', 'anak tangga curam', 'anak tangga enak', 'anak tangga lelah', 'anak tangga lewat', 'anti', 'anti foto', 'anti foto pandang', 'arah', 'arah bedugul', 'arah bedugul belok', 'arah kanan', 'arah kanan tiga', 'areal', 'areal sawah', 'areal sawah kalah', 'arus', 'arus karang', 'arus kar

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Max_feature

In [ ]:
vectorizer_max_features = CountVectorizer(analyzer='word', ngram_range=(1,3), max_features = 100)
bow_matrix_max_features = vectorizer_max_features.fit_transform(reviews2)

In [ ]:
print(vectorizer_max_features.get_feature_names())
print(bow_matrix_max_features.toarray())

### Max_df & Min_df

In [ ]:
vectorizer_max_min = CountVectorizer(analyzer='word', ngram_range=(1,3), max_df =3, min_df = 2)
bow_matrix_max_min = vectorizer_max_min.fit_transform(preprocessed_corpus)

In [ ]:
print(vectorizer_max_min.get_feature_names())
print(bow_matrix_max_min.toarray())

## TF-IDF

In [ ]:
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(preprocessed_corpus)

In [ ]:
print(vectorizer.get_feature_names())
print(tf_idf_matrix.toarray())
print("\nThe shape of the TF-IDF matrix is: ", tf_idf_matrix.shape)